In [1]:
from keras.optimizers import *
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils import np_utils

import os,sys
import numpy as np
import pandas as pd
import cv2,time,glob,math
import matplotlib.pyplot as plt # plt 用于显示图片
from tqdm import tqdm #进度条

Using TensorFlow backend.


In [2]:
#设置一些基本参数
train_dir = 'train'
valid_dir = 'validation'
test_dir = 'test2'
nb_train_samples = sum([len(x) for _, _, x in os.walk(train_dir)])
nb_valid_samples = sum([len(x) for _, _, x in os.walk(valid_dir)])
nb_test_samples = sum([len(x) for _, _, x in os.walk(test_dir)])
nb_categories = 10
epochs = 5
labels = [" normal driving","texting - right","talking on the phone - right","texting - left",
          "talking on the phone - left","operating the radio",
          "drinking"," reaching behind","hair and makeup","talking to passenger"]

In [3]:
print nb_valid_samples

2422


In [4]:
from keras import backend as K
K.clear_session()

In [23]:
#set training parameters
model_name = 'InceptionV3'
fine_tune_layer = 133
optimizer = 'Adam'
learning_rate = 1e-6
augmentation = 'yes'
valid_set = 'p024_p026'
batch_size = 32
img_width, img_height = 400, 300

In [7]:
# 利用Keras的ImageDataGenerator对图片进行预处理
from keras.applications.inception_v3 import preprocess_input

train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=10) 
#考虑到体型、姿势略有不同，设置少许的旋转、放大缩小和平移。不做翻转） 

datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
#验证集和测试集无须进行上述处理，保留原样

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(img_width,img_height),  
        shuffle=True,
        batch_size=batch_size,
        class_mode='categorical')  


validation_generator = datagen.flow_from_directory(
        valid_dir,
        target_size=(img_width,img_height),
        shuffle=True,
        batch_size=batch_size,
        class_mode='categorical')

test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(img_width,img_height),
        shuffle=False,
        batch_size=batch_size,
        class_mode='categorical')

Found 20002 images belonging to 10 classes.
Found 2422 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [8]:
inputs = Input(shape=(img_width, img_height, 3))
base_model = InceptionV3(input_tensor=inputs, weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs, x)

print 'total layer count: ', len(base_model.layers),
print '| fine tune layer: ', fine_tune_layer

for i in range(fine_tune_layer):
    model.layers[i].trainable = False

total layer count:  311 | fine tune layer:  133


In [24]:
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
result = model.fit_generator(train_generator,
                   steps_per_epoch=nb_train_samples//batch_size+1,
                   epochs=1,
                   validation_data=validation_generator,
                   validation_steps=nb_valid_samples//batch_size+1)

Epoch 1/1
626/626 [==============================] - 789s - loss: 0.0073 - acc: 0.9982 - val_loss: 0.1821 - val_acc: 0.9348


In [21]:
model.save_weights('models/IncpetionV3_Adam_1e-5_p024_p026-400_300_tune_133.h5')

In [25]:
#write results, use'a' to countinue writing
import csv
print 'model  | layer|  opt  |  l-rate |  aug  |    driver   |batch | epo |  t_acc  |  t_loss |  v_acc  | v_loss '
print '----------------------------------------------------------------------------------------------------------'
with open("result201102.csv","a") as csvfile: 
    writer = csv.writer(csvfile)
    for i in range(len(result.epoch)):
        writer.writerow([model_name, fine_tune_layer, optimizer, learning_rate, augmentation, 
                         valid_set, batch_size, result.epoch[i], '%.3f' % result.history['acc'][i],  
                         '%.3f' % result.history['loss'][i], '%.3f' % result.history['val_acc'][i],
                         '%.3f' % result.history['val_loss'][i]])
        print model_name,' | ', fine_tune_layer,' | ', optimizer, ' | ',learning_rate,' | ', augmentation,
        print ' | ',valid_set, ' | ', batch_size, ' | ', result.epoch[i], ' | ', 
        print '%.3f' % result.history['acc'][i],' | ','%.3f' % result.history['loss'][i], ' | ',
        print '%.3f' % result.history['val_acc'][i],' | ','%.3f' % result.history['val_loss'][i]

model  | layer|  opt  |  l-rate |  aug  |    driver   |batch | epo |  t_acc  |  t_loss |  v_acc  | v_loss 
----------------------------------------------------------------------------------------------------------
InceptionV3  |  133  |  Adam  |  1e-06  |  yes  |  p024_p026  |  32  |  0  |  0.998  |  0.007  |  0.935  |  0.182


In [26]:
#load csv and show results
import csv
with open('result201102.csv',"r") as csvfile:
    reader = csv.reader(csvfile)
    #这里不需要readlines
    for line in reader:
        for i in range(len(line)):
            print line[i]," | ",
        print

model  |  fine_tune_layer  |  optimizer  |  learning_rate  |  augmentation  |  valid_set  |  batch_size  |  epochs  |  train_acc  |  train_loss  |  valid_acc  |  valid_loss  |  train_time  | 
ResNet50  |  109  |  Adam  |  1.00E-05  |  yes  |  p021_p022  |  64  |  0  |  0.445  |  1.642  |  0.624  |  1.063  |    | 
ResNet50  |  109  |  Adam  |  1.00E-05  |  yes  |  p021_p022  |  64  |  0  |  0.958  |  0.167  |  0.87  |  0.424  |    | 
ResNet50  |  109  |  Adam  |  1.00E-05  |  yes  |  p021_p022  |  64  |  0  |  0.976  |  0.096  |  0.878  |  0.401  |    | 
ResNet50  |  109  |  Adam  |  1.00E-05  |  yes  |  p021_p022  |  64  |  1  |  0.985  |  0.06  |  0.876  |  0.387  |    | 
ResNet50  |  109  |  Adam  |  1.00E-05  |  yes  |  p021_p022  |  64  |  0  |  0.976  |  0.096  |  0.878  |  0.401  |    | 
ResNet50  |  109  |  Adam  |  1.00E-05  |  yes  |  p021_p022  |  64  |  1  |  0.985  |  0.06  |  0.876  |  0.387  |    | 
ResNet50  |  109  |  Adam  |  1.00E-06  |  yes  |  p021_p022  |  64  |  0

In [27]:
y_pred = model.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)

2492/2492 [==============================] - 1800s  


In [30]:
l = list()
for i, fname in enumerate(test_generator.filenames):
    name = fname[fname.rfind('/')+1:]
    line = y_pred[i]
    line = line.tolist()
    line.insert(0,name)
    l.append( line )

l = np.array(l)
data = {'img': l[:,0]}
for i in range(10):
    data["c%d"%i] = l[:,i+1]
df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
df = df.sort_values(by='img')
df.to_csv('5_incpetionV3_p024_p026_l133.csv', index=None, float_format='%.3f')
print("submission saved")

submission saved


In [32]:
pre_submission = pd.read_csv('5_incpetionV3_p024_p026_l133.csv')
clip_submission = pre_submission.drop('img',axis =1)
clip_submission = clip_submission.clip(0.001,1)
clip_submission.insert(0, 'img', pre_submission['img'])
clip_submission.to_csv('clipped_5_incpetionV3_p024_p026_l133.csv', index=False)